In [1]:
# Initialisation and loadup
from http import server
import pandas as pd
import numpy as np
import copy
import os
import time
import re
p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+' #REparser

import selenium as sl
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driverpath=ChromeDriverManager().install()

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [2]:
np.load('./s1scrapedata/scrapedpageindex.npy')

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101.])

In [3]:
# initialise the pandas dataframe
columns = ["Index_Link", "Legal_Name", "Town/Suburb","State", "Status", "Size","ABN","Website","Revenue","Expenses","Cause/s","Last_report_date","ref_religion"]

#list off lowercase religous words
religious_buzzwords = ['church','priest','christian','islam', 'muslim','religion','religious','evangelical','parish','jehovah','worship', 'judaism', ' god ',' God ',"god's",'gospel', 'bible', 'biblical']

#extend religious word dictionary
CapWords=[]
CAPWORDS=[]
for word in religious_buzzwords:
    CapWords.append(word.capitalize())
    CAPWORDS.append(word.upper())
religious_buzzwords.extend(w for w in CapWords)
religious_buzzwords.extend(w for w in CAPWORDS)

    

In [4]:
#get list of files to rescrape
files = os.listdir(os.getcwd()+'/s1scrapedata')
files = [i for i in files if 'topage' in i]
files

['S1scrape_topage103.csv',
 'S1scrape_topage15.csv',
 'S1scrape_topage30.csv',
 'S1scrape_topage45.csv',
 'S1scrape_topage60.csv',
 'S1scrape_topage75.csv',
 'S1scrape_topage90.csv']

In [5]:
def open_drivers(num_drivers=5):
    drivers =[]
    for i in range(num_drivers):
        drivers.append(webdriver.Chrome(service=Service(driverpath)))
    return drivers

In [6]:
def getpageurl(c,df):
    url = df.iloc[c,1]
    return url

In [7]:
def extract_page_data(d):
    try:
        WebDriverWait(d,35).until(EC.presence_of_element_located((By.XPATH,"//p[contains(text(),': $')]")))
        rev_and_expenses = d.find_elements(By.XPATH,"//p[contains(text(),': $')]")
        revenue = float(re.findall(p, rev_and_expenses[0].text)[0].replace(",",""))
        expenses =float(re.findall(p, rev_and_expenses[1].text)[0].replace(",",""))
    except:
        revenue = None
        expenses = None
        print('revecpenses failed')

    
    # last report date
    try:
        lastreport = d.find_elements(by=By.XPATH, value="//*[contains(text(),'Last reported:')]/..//dd")[0].text
    except:
        lastreport = None
        print("date failed")

    try:
        causes = d.find_elements(by=By.XPATH, value="//*[contains(text(),'Who the charity helps:')]/..//dd")[0].text.replace("\n","; ")
    except:
        causes = None
        print('causes failed')

    try:
        website = d.find_element(by=By.XPATH, value="//*[contains(text(),'Website:')]/..//a").get_attribute('href')
    except:
        website = None
        print("no website/failed")

    religious_ref = 0
    page_text = d.find_element(By.XPATH, '//*').text
    for word in religious_buzzwords:
        if word in page_text:
            religious_ref+=1
            break
        else:
            continue

    
    page_data = [website,revenue,expenses,causes,lastreport,religious_ref]
    return page_data #in form [Website,Revenue,Expenses,Cause/s,Last_report_date,ref_religion]

In [8]:
def scrapecsv(file,drivers):
    filedf = pd.read_csv('./s1scrapedata/'+file)
    nocharities = filedf.shape[0]-1 #no. charities in csv
    tempdf = pd.DataFrame([],columns=columns)
    charity = 0


    try:
        ScrapedCharities = np.load('./s2scrapeddata/scrapedcharityindex'+file+'.npy')
    except:
        ScrapedCharities = np.array([])
    
    
    try:
        #navigate into pages and scrape

        while charity < nocharities:
            #load up sites
            charcopy = copy.deepcopy(charity)
            for d in drivers:
                while charcopy in ScrapedCharities:
                    charcopy+=1
                if charcopy < nocharities:
                    d.get(getpageurl(charcopy,filedf))        
                    charcopy+=1

            ## scraping
            for D in drivers:
                try:
                    while charity in ScrapedCharities:
                        charity +=1
                    page_data = extract_page_data(d=D)
                    tempdf.loc[charity,:] = filedf.loc[charity,:]
                    tempdf.loc[charity,['website','revenue','expenses','causes','lastreport','religious_ref']] = page_data
                    ScrapedCharities=np.append(ScrapedCharities, charity)

                    if (charity+1) % 300 == 0:
                        np.save('./s2scrapedata/scrapedcharityindex'+file+'.npy',ScrapedCharities)
                        tempdf.to_csv('./s2scrapedata/S2scrape_tochar__'+file+'__'+str(charity+1)+'__.csv')
                        tempdf = pd.DataFrame([],columns=columns)

                    charity += 1
                    print("Progress: "+str(charity)+"/"+str(nocharities+1))
                except:
                    print("maybe the drivers are continuing to iterate over entries that don't exist?")

        np.save('./s2scrapedata/scrapedcharityindex'+file+'.npy',ScrapedCharities)
        tempdf.to_csv('./s2scrapedata/S2scrape_tochar__'+file+'__'+str(charity+1)+'__.csv')
        tempdf = pd.DataFrame([],columns=columns)


        
        
        

    except Exception as e:
        for d in drivers:
            d.quit()
        print('error in scrapeCSV')
        raise e    




In [9]:
def run_scraper():
    
    drivers = open_drivers(3)
    try:
        try:
            fileprogress = np.load('./s2scrapedata/scrapedfileindex.npy')
        except:
            fileprogress = np.array([])

        for file in files:
            if file in fileprogress:
                continue
            scrapecsv(file,drivers)
            fileprogress = np.append(fileprogress, file)
            np.save('./s2scrapedata/scrapedfileindex.npy',fileprogress)

        #kill drivers
        for d in drivers:
            d.quit()    
    
    except Exception as e:
        for d in drivers:
            d.quit()
        raise e


In [10]:
run_scraper()

C:\Users\61457\AppData\Local\Temp\ipykernel_39884\938399324.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if file in fileprogress:


Progress: 1/1200
Progress: 2/1200
Progress: 3/1200
Progress: 4/1200
Progress: 5/1200
Progress: 6/1200
Progress: 7/1200
no website/failed
Progress: 8/1200
Progress: 9/1200
Progress: 10/1200
Progress: 11/1200
Progress: 12/1200
Progress: 13/1200
no website/failed
Progress: 14/1200
Progress: 15/1200
Progress: 16/1200
Progress: 17/1200
Progress: 18/1200
Progress: 19/1200
Progress: 20/1200
Progress: 21/1200
Progress: 22/1200
Progress: 23/1200
Progress: 24/1200
Progress: 25/1200
Progress: 26/1200
Progress: 27/1200
Progress: 28/1200
Progress: 29/1200
Progress: 30/1200
Progress: 31/1200
Progress: 32/1200
Progress: 33/1200
Progress: 34/1200
Progress: 35/1200
Progress: 36/1200
Progress: 37/1200
Progress: 38/1200
Progress: 39/1200
Progress: 40/1200
no website/failed
Progress: 41/1200
Progress: 42/1200
Progress: 43/1200
Progress: 44/1200
Progress: 45/1200
Progress: 46/1200
Progress: 47/1200
Progress: 48/1200
Progress: 49/1200
Progress: 50/1200
Progress: 51/1200
Progress: 52/1200
Progress: 53/1200
P

In [ ]:
# page_data = ['website','revenue','expenses','causes','lastreport','religious_ref']
# file='S1scrape_topage104.csv'
# filedf = pd.read_csv('./s1scrapedata/'+file)
# #tempdf = pd.DataFrame([],columns=columns)
# tempdf.loc[1,:]=filedf.loc[1,:]
# tempdf.iloc[1,[7,8,9,10,11,12]] = page_data
# tempdf